In [30]:
#Script para gerar tabela Forma Pagamento

import pandas as pd
from datetime import date
from sqlalchemy import create_engine, VARCHAR, DATE, INTEGER, text
import numpy as np
import unicodedata

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"
conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

# Função para remover espaços desnecessários em todos os tipos de dados
def strip_all_types(col):
    # Converte a coluna para string, remove espaços extras e aplica strip
    col_stripped = col.astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
    
    # Converte de volta para o tipo original da coluna
    if np.issubdtype(col.dtype, np.integer):  # Se for inteiro
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif np.issubdtype(col.dtype, np.floating):  # Se for float
        return pd.to_numeric(col_stripped, errors='coerce').astype(col.dtype)
    elif np.issubdtype(col.dtype, np.datetime64):  # Se for data
        return pd.to_datetime(col_stripped, errors='coerce')
    else:  # Se for string ou outro tipo
        return col_stripped


lista_forma_pagamento = ["Dinheiro", "Pix", "Cartao Debito","Cartao Credito a vista", 
                         "Cartao Credito parcelado", "Vale-Alimentacao", "Cheque"]

tabela = "FORMA_PAGAMENTO"
coluna = "FORMA"

with engine.connect() as conn:
    query = text(f"SELECT {coluna} FROM {tabela}")
    lista_valores_existentes = pd.read_sql(query, conn)[coluna].tolist()

    df = pd.DataFrame({'coluna1': lista_forma_pagamento})

    df = df.apply(strip_all_types)
  
    lista_forma_pagamento = df['coluna1'].tolist()
    
    lista_valores_para_adicionar = [forma for forma in lista_forma_pagamento if forma not in lista_valores_existentes]

    if lista_valores_para_adicionar:
         
        df_FORMA_PAGAMENTO = pd.DataFrame({
                                            'FORMA':         lista_valores_para_adicionar,
                                            'DATA_CRIACAO':  date.today().strftime('%d/%m/%Y'),
                                            'DATA_ATLZ':     " ",
                                            'VERSAO':        1,
        })
        
        tipo_dados_forma_pagamento = {
                                'FORMA':         VARCHAR(50),
                                'DATA_CRIACAO':  DATE,
                                'DATA_ATLZ':     DATE,
                                'VERSAO':        INTEGER,

        }
        
        #Limpa os espacos desnecessarios
        df_FORMA_PAGAMENTO = df_FORMA_PAGAMENTO.apply(strip_all_types)
        display(df_FORMA_PAGAMENTO)
        #envia a tabela
        df_FORMA_PAGAMENTO.to_sql(name= tabela, con=engine,if_exists='append',index=False,dtype=tipo_dados_forma_pagamento)
        
        print("Dados inseridos com sucesso.")
    else:
        print("Nenhum valor adicionado.")    

engine.dispose()



,FORMA,DATA_CRIACAO,DATA_ATLZ,VERSAO
0,Dinheiro,16/03/2025,,1
1,Pix,16/03/2025,,1
2,Cartao Debito,16/03/2025,,1
3,Cartao Credito a vista,16/03/2025,,1
4,Cartao Credito parcelado,16/03/2025,,1
5,Vale-Alimentacao,16/03/2025,,1
6,Cheque,16/03/2025,,1


Dados inseridos com sucesso.


In [ ]:
#Script para adicionar valor por lista em todas as colunas da tabela Forma Pagamento

import pandas as pd
from datetime import date
from sqlalchemy import create_engine, VARCHAR, DATE, INTEGER, text

# Configurações de conexão ao banco de dados
server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"
conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)


# Variáveis para armazenar o nome da tabela e coluna
tabela = "FORMA_PAGAMENTO"  # Nome da tabela

# Listas de entrada para as colunas
lista_entrada_FORMA = ["TESTE"]
lista_entrada_DATA_CRIACAO = ['2020/01/01']
lista_entrada_DATA_ATLZ = [" "]
lista_entrada_VERSAO = [5]

# Listas de entrada para as colunas
lista_entrada = {
    'FORMA': lista_entrada_FORMA,  # Pode estar vazia
    'DATA_CRIACAO': lista_entrada_DATA_CRIACAO,  # Pode estar vazia
    'DATA_ATLZ': lista_entrada_DATA_ATLZ,  # Pode estar vazia
    'VERSAO': lista_entrada_VERSAO  # Pode estar vazia
}

# Verifica valores existentes no banco de dados
with engine.connect() as conn:
    lista_valores_para_adicionar = []
    
    # Itera sobre as colunas e suas listas de entrada
    for coluna, valores in lista_entrada.items():
        if valores:  # Verifica se a lista não está vazia
            query = text(f"SELECT {coluna} FROM {tabela}")
            lista_valores_existentes = pd.read_sql(query, conn)[coluna].tolist()
            lista_valores_para_adicionar.extend([valor for valor in valores if valor not in lista_valores_existentes])

    # Se houver valores novos, cria o DataFrame e insere no banco de dados
    if lista_valores_para_adicionar:
        #Cria um novo DataFrame com os valores a serem adicionados
        df_FORMA_PAGAMENTO = pd.DataFrame(lista_entrada)

        #Define os tipos de dados para as colunas
        tipo_dados_forma_pagamento = {
            'FORMA': VARCHAR(50),
            'DATA_CRIACAO': DATE,
            'DATA_ATLZ': DATE,
            'VERSAO': INTEGER
        }

        #Insere os dados no banco de dados
        df_FORMA_PAGAMENTO.to_sql(name=tabela, con=engine, if_exists='append', index=False, dtype=tipo_dados_forma_pagamento)
        print("Dados inseridos com sucesso.")
    else:
        print("Nenhum valor adicionado.")

#Fecha a conexão com o banco de dados
engine.dispose()

In [ ]:
#Script para adicionar valor por lista em todas as colunas da tabela com verificacao de erro


import pandas as pd
from datetime import date
from sqlalchemy import create_engine, VARCHAR, DATE, INTEGER, text
import sys  # Importa o módulo sys para usar sys.exit()

# Configurações de conexão ao banco de dados
server = "DESKTOP-HV85ADA\\SERVERSQL22"
bancodados = "TESTE"
driver = "ODBC Driver 17 for SQL Server"
conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

# Data atual
DATAHOJE = date.today().strftime('%d/%m/%Y')

# Variáveis para armazenar o nome da tabela
tabela = "FORMA_PAGAMENTO"  # Nome da tabela

# Listas de entrada para as colunas
lista_entrada_FORMA = ["sniuvn"]  # Exemplo de dados
lista_entrada_CIDADE = ["svfvv"]  # Exemplo de dados
lista_entrada_ESTADO = ["kl"]  # Exemplo de dados
lista_entrada_DATA_CRIACAO = [DATAHOJE]  # Exemplo de dados
lista_entrada_DATA_ATLZ = [""]  # Exemplo de dados
lista_entrada_VERSAO = [1]  # Exemplo de dados

# Dicionário para armazenar as listas de entrada
lista_entrada = {
    'FORMA': lista_entrada_FORMA,
    'CIDADE': lista_entrada_CIDADE,
    'ESTADO': lista_entrada_ESTADO,
    'DATA_CRIACAO': lista_entrada_DATA_CRIACAO,
    'DATA_ATLZ': lista_entrada_DATA_ATLZ,
    'VERSAO': lista_entrada_VERSAO
}

try:
    # Verifica se as listas FORMA, CIDADE e ESTADO têm o mesmo tamanho
    tamanho_FORMA = len(lista_entrada_FORMA)
    tamanho_CIDADE = len(lista_entrada_CIDADE)
    tamanho_ESTADO = len(lista_entrada_ESTADO)

    if tamanho_FORMA != tamanho_CIDADE or tamanho_FORMA != tamanho_ESTADO:
        raise ValueError("As listas 'lista_entrada_FORMA', 'lista_entrada_CIDADE' e 'lista_entrada_ESTADO' devem ter o mesmo tamanho.")

    # Verifica valores existentes no banco de dados
    with engine.connect() as conn:
        # Dicionário para armazenar valores que não existem no banco de dados
        dados_para_adicionar = {coluna: [] for coluna in lista_entrada.keys()}

        # Verifica as colunas FORMA, CIDADE e ESTADO
        for coluna in ['FORMA', 'CIDADE', 'ESTADO']:
            if lista_entrada[coluna]:  # Verifica se a lista não está vazia
                query = text(f"SELECT {coluna} FROM {tabela}")
                lista_valores_existentes = pd.read_sql(query, conn)[coluna].tolist()
                # Filtra valores que não existem no banco de dados
                dados_para_adicionar[coluna] = [valor for valor in lista_entrada[coluna] if valor not in lista_valores_existentes]

        # Verifica se as listas filtradas têm o mesmo tamanho
        tamanho_filtrado_FORMA = len(dados_para_adicionar['FORMA'])
        tamanho_filtrado_CIDADE = len(dados_para_adicionar['CIDADE'])
        tamanho_filtrado_ESTADO = len(dados_para_adicionar['ESTADO'])

        if tamanho_filtrado_FORMA != tamanho_filtrado_CIDADE or tamanho_filtrado_FORMA != tamanho_filtrado_ESTADO:
            raise ValueError("Após a filtragem, as listas 'FORMA', 'CIDADE' e 'ESTADO' ficaram com tamanhos diferentes. Verifique os valores existentes no banco de dados.")

        # Verifica se há valores novos para adicionar
        if any(dados_para_adicionar.values()):
            # Cria um novo DataFrame apenas com os valores que não existem no banco de dados
            df_FORMA_PAGAMENTO = pd.DataFrame({
                'FORMA': dados_para_adicionar['FORMA'],
                'CIDADE': dados_para_adicionar['CIDADE'],
                'ESTADO': dados_para_adicionar['ESTADO'],
                'DATA_CRIACAO': lista_entrada['DATA_CRIACAO'][:len(dados_para_adicionar['FORMA'])],  # Ajusta o tamanho
                'DATA_ATLZ': lista_entrada['DATA_ATLZ'][:len(dados_para_adicionar['FORMA'])],  # Ajusta o tamanho
                'VERSAO': lista_entrada['VERSAO'][:len(dados_para_adicionar['FORMA'])]  # Ajusta o tamanho
            })

            # Define os tipos de dados para as colunas
            tipo_dados_forma_pagamento = {
                'FORMA': VARCHAR(50),
                'CIDADE': VARCHAR(50),
                'ESTADO': VARCHAR(2),
                'DATA_CRIACAO': DATE,
                'DATA_ATLZ': DATE,
                'VERSAO': INTEGER
            }

            # Insere os dados no banco de dados
            df_FORMA_PAGAMENTO.to_sql(name=tabela, con=engine, if_exists='append', index=False, dtype=tipo_dados_forma_pagamento)
            print("Dados inseridos com sucesso.")
        else:
            print("Nenhum valor adicionado.")

except ValueError as e:
    # Exibe apenas a mensagem de erro no console
    print(f"Erro: {e}")
    sys.exit(1)  # Encerra o script com código de erro 1

finally:
    # Fecha a conexão com o banco de dados
    engine.dispose()

Erro: Após a filtragem, as listas 'FORMA', 'CIDADE' e 'ESTADO' ficaram com tamanhos diferentes. Verifique os valores existentes no banco de dados.


SystemExit: 1